# Import Libraries

In [3]:
# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

# Librerias de ayuda
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print(tf.__version__)

2.9.1


# Import Dataset from csv file

In [6]:
df = pd.read_csv(r'Dataset\dataset-song-genre.csv')
print(df)

                     track_id                       artists  \
0      3txHZTpGeVASXa5ARwoRX6                        Frejat   
1      2e7LBdeiidf0ZC5Qj52u0G                         STU48   
2      0hlh787aY52rcNvJHnE0Ml             Mike NRG;Weapon X   
3      3s067jTdOv8wnuUHeXdtjT        NICO Touches the Walls   
4      4oE62AqzUuiwV09HGxiMlu       Edvard Grieg;Una Bourne   
...                       ...                           ...   
91195  0Y1hw2GXV8npQI7C21ZIT1                      Surfaces   
91196  43QSsuxazI8xms29SYCtxh                 KANA-BOON;もっさ   
91197  7nbj09isjVmgNYJ7eKBzyt                    Zach Bryan   
91198  5bVUUTXB0Dg7EzuSZC57ek  Joey Montana;Sebastian Yatra   
91199  2RjWCcfCOMzKeRJzeoe4h6            Noisia;The Upbeats   

                                album_name  \
0            Frejat Ao Vivo No Rock In Rio   
1                       無謀な夢は覚めることがない【劇場盤】   
2         Lost in Dreams (Q-Base OST 2008)   
3                                     オーロラ   
4      Mozart

In [7]:
df.shape

(91200, 20)